In [2]:
import random
import random as rande
import numpy as np
import numpy
import math
import matplotlib.pyplot as plt

from numpy.random import uniform, normal, randint, rand
from numpy import exp, pi, sin, cos, zeros, minimum, maximum, abs, where, sign, mean, stack
from numpy import min as np_min
from numpy import max as np_max
from copy import deepcopy

from numpy import where, clip, logical_and, maximum, minimum, power, sin, abs, pi, sqrt, sign, ones, ptp, min, sum, array, ceil, multiply, mean
from numpy.random import uniform, random, normal, choice
from math import gamma

import time
import csv

ID_MIN_PROB = 0
ID_MAX_PROB = -1
ID_POS = 0  
ID_FIT = 1

EPSILON = 10E-10
DEFAULT_BATCH_IDEA = False
DEFAULT_BATCH_SIZE = 10
DEFAULT_LB = -1
DEFAULT_UB = 1

class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual = []
        self.convergence = []
        self.optimizer = ""
        self.objfname = ""
        self.startTime = 0
        self.endTime = 0
        self.executionTime = 0
        self.lb = 0
        self.ub = 0
        self.dim = 0
        self.popnum = 0
        self.maxiers = 0




def WMFO(objf, lb, ub, dim, SearchAgents_no, Max_iter,verbose=False):

    Convergence_curve = numpy.zeros(Max_iter)

    soll = solution()
    timerStart = time.time()
    soll.startTime = time.strftime("%Y-%m-%d-%H-%M-%S")

 
    if not isinstance(lb, list):
        lb = [lb] * dim
    if not isinstance(ub, list):
        ub = [ub] * dim    
    epochs = Max_iter
    pop_size = SearchAgents_no

    epochs = Max_iter
    problem_size = dim
    ID_WEI = 2
 

    pop_moths = [create_solution(lb,ub,objf,problem_size) for _ in range(SearchAgents_no)]
   
    pop_flames, g_best = get_sorted_pop_and_global_best_solution(pop=pop_moths, id_fit=ID_FIT, id_best=ID_MIN_PROB)     # Eq.(2.6)
    g_worst = pop_flames[-1]
    problem_size = dim
    
    for epoch in range(epochs):
            

            num_flame = round(SearchAgents_no - (epoch + 1) * ((SearchAgents_no - 1) / epochs))

            
            a = -1 + (epoch + 1) * ((-1) / epochs)
            for i in range(SearchAgents_no):
                r = rande.random()

                distance_to_flame = abs(pop_flames[i][ID_POS] - pop_moths[i][ID_POS])
                t = (a - 1) * uniform(0, 1, problem_size) + 1
                b = 1
                
                temp_1 = distance_to_flame * exp(b * t) * cos(t * 2 * pi) + pop_flames[i][ID_POS]


                temp_2 = distance_to_flame * exp(b * t) * cos(t * 2 * pi) + g_best[ID_POS]

                list_idx = i * ones(problem_size)
                pos_new = where(list_idx < num_flame, temp_1, temp_2)

                
                r1 = rande.random()  
                r2 = rande.random() 
                a1 = 1 - epoch * ((1) / epochs)
                r4 = rande.random()
                A =  2 * a1 * r1 - a1  
                C = 2 * r2 
                Xnew = r4*g_best[ID_POS] - A * abs(C*g_best[ID_POS]-pos_new)+ (1-r4)* g_worst[ID_POS]
            
                if objf(Xnew)< objf(pos_new):
                    pos_new=Xnew                  
                    pos_new = amend_position_faster(pos_new,lb,ub)
                
                fit_new = get_fitness_position(objf,pos_new)
                if fit_new < pop_moths[i][ID_FIT]:
                    pop_moths[i] = [pos_new, fit_new]


            pop_flames = pop_flames + pop_moths
            pop_flames, g_best = update_sorted_population_and_global_best_solution(pop_flames, ID_MIN_PROB, g_best)
            pop_flames = pop_flames[:SearchAgents_no]
            g_worst = pop_flames[-1]

            Convergence_curve[epoch] = g_best[ID_FIT]
            if verbose:
                print("> Epoch: {}, Best fit: {}".format(epoch + 1, g_best[ID_FIT]))

    print(f"{g_best[ID_FIT]:e}")

    timerEnd = time.time()
    soll.endTime = time.strftime("%Y-%m-%d-%H-%M-%S")
    soll.executionTime = timerEnd - timerStart
    soll.convergence = Convergence_curve
    soll.optimizer = "WMFO"
    soll.objfname = objf.__name__
    soll.best = g_best[ID_FIT]    
    soll.bestIndividual = g_best[ID_POS]
    return soll

def update_sorted_population_and_global_best_solution(pop=None, id_best=None, g_best=None):
    sorted_pop = sorted(pop, key=lambda temp: temp[ID_FIT])
    current_best = sorted_pop[id_best]
    g_best = deepcopy(current_best) if current_best[ID_FIT] < g_best[ID_FIT] else deepcopy(g_best)
    return sorted_pop, g_best
def amend_position_faster(position, lb, ub):
    #return clip(position, lb, ub)
    return np.where(np.logical_and(lb <= position, position <= ub), position, np.random.uniform(lb, ub))

def create_solution(lb , ub, obj_func,dim,minmax=0):
    position = uniform(lb,ub)
    fitness = get_fitness_position(obj_func, position=position, minmax=minmax)
    weight = zeros(dim)
    return [position, fitness,weight]

def get_sorted_pop_and_global_best_solution(pop=None, id_fit=None, id_best=None):
    sorted_pop = sorted(pop, key=lambda temp: temp[id_fit])
    return sorted_pop, deepcopy(sorted_pop[id_best])


def get_fitness_position(obj_func,position, minmax=0):
    fit_new = obj_func(position)
    return fit_new if minmax == 0 else 1.0 / (fit_new + EPSILON)
def weighted_lehmer_mean(list_objects, list_weights):
    up = list_weights * list_objects ** 2
    down = list_weights * list_objects
    return sum(up) / sum(down)

def update_global_best_solution(pop=None, id_best=None, g_best=None):
    sorted_pop = sorted(pop, key=lambda temp: temp[ID_FIT])
    current_best = sorted_pop[id_best]
    return deepcopy(current_best) if current_best[ID_FIT] < g_best[ID_FIT] else deepcopy(g_best)
def get_global_best_solution(pop=None, id_fit=None, id_best=None):
    sorted_pop = sorted(pop, key=lambda temp: temp[id_fit])
    return deepcopy(sorted_pop[id_best])
